In [1]:
import pandas as pd
import numpy as np
from guizero import *
import PIL

In [2]:
try:
    df=pd.read_feather("NN_INPUT.feather")
except Exception as E:
    print(E)
    df=pd.read_feather("NN_INPUT.backup.feather")

In [3]:
df.head(10)

,Path,FUM,EXP,INAC
0,2000/abr/p0423001.jpg,0.0,0.0,1.0
1,2000/abr/p0423002.jpg,1.0,0.0,0.0
2,2000/abr/p0423003.jpg,1.0,0.0,0.0
3,2000/abr/p0424001.jpg,1.0,0.0,0.0
4,2000/abr/p0424002.jpg,1.0,0.0,0.0
5,2000/abr/p0425001.jpg,1.0,0.0,0.0
6,2000/abr/p0425002.jpg,1.0,0.0,0.0
7,2000/abr/p0427001.jpg,1.0,0.0,0.0
8,2000/abr/p0427002.jpg,0.0,0.0,0.0
9,2000/abr/p0428001.jpg,0.0,0.0,0.0


In [4]:
mcl_df=pd.read_csv("model_misclassifications.csv",index_col=0)
mcl_df.set_index("RealID",drop=True,inplace=True)
mcl_df["Written"]=np.zeros(mcl_df.shape[0])
mcl_df

,ID,RealClass,PredictedClass,RealClassID,PredictedClassID,Certainty,Written
RealID,,,,,,,
9577,1843,FUM,EXP+FUM,2,3,100.0,0.0
18857,19638,FUM,EXP+FUM,2,3,100.0,0.0
11721,7388,UNK,FUM,0,2,100.0,0.0
18854,1027,EXP,EXP+FUM,1,3,100.0,0.0
9338,19858,EXP+FUM,FUM,3,2,100.0,0.0
...,...,...,...,...,...,...,...
10557,9385,EXP+FUM,-1,3,-1,0.0,0.0
13191,5130,EXP+FUM,-1,3,-1,0.0,0.0
776,20157,EXP+FUM,-1,3,-1,0.0,0.0


In [5]:
def get_class(row):
    _,F,E,N=row
    L="UNK"
    if F>0:
        if E>0:
            L="EXP+FUM"
        else:
            L="FUM"
    elif E>0:
        L="EXP"
    elif N>0:
        L="INA"
    return L
def get_iclass(df,idx):
    row=df.iloc[idx]
    return get_class(row)

In [6]:
get_iclass(df,9577)

'FUM'

In [7]:
def set_iclass(df,idx,class_):
    row=df.iloc[idx]
    path=row["Path"]
    if(class_=="INAC"):
        df.iloc[idx]=[path,0.0,0.0,1.0]
    elif(class_=="FUM"):
        df.iloc[idx]=[path,1.0,0.0,0.0]
    elif(class_=="EXP"):
        df.iloc[idx]=[path,0.0,1.0,0.0]
    elif(class_=="FUM+EXP"):
        df.iloc[idx]=[path,1.0,1.0,0.0]
    elif(class_=="UNK"):
        df.iloc[idx]=[path,0.0,0.0,0.0]
#set_iclass(df,9577,"FUM")
print(df.iloc[9577])

Path    2014/oct/p1030149.jpg
FUM                       1.0
EXP                       0.0
INAC                      0.0
Name: 9577, dtype: object


In [8]:
mcl_df.iloc[0]

ID                     1843
RealClass               FUM
PredictedClass      EXP+FUM
RealClassID               2
PredictedClassID          3
Certainty             100.0
Written                 0.0
Name: 9577, dtype: object

In [9]:
remaining_ct=0

In [10]:
def get_current_row():
    global remaining_ct
    to_fix=df.iloc[mcl_df.index]
    current_labels=[get_class(to_fix.iloc[i]) for i in range(to_fix.shape[0])]
    remaining=mcl_df[mcl_df["RealClass"]==current_labels]
    remaining=remaining[mcl_df["Written"]!=1]
    
    remaining_ct=len(remaining)
    return remaining.iloc[0],"images/"+df.iloc[remaining.index[0]]["Path"]


In [11]:
row,path=get_current_row()
print(row,path)

ID                     1843
RealClass               FUM
PredictedClass      EXP+FUM
RealClassID               2
PredictedClassID          3
Certainty             100.0
Written                 0.0
Name: 9577, dtype: object images/2014/oct/p1030149.jpg


/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]


In [12]:
df.iloc[row.name]

Path    2014/oct/p1030149.jpg
FUM                       1.0
EXP                       0.0
INAC                      0.0
Name: 9577, dtype: object

In [13]:
def get_size(path:str) -> tuple[int, int]:
    im = PIL.Image.open(path)
    return im.size

In [14]:
row,path=get_current_row()
w,h=get_size(path)
app = App(
    height=h+25*6,
    width=w, 
    bg="white"
)


pic = Picture(app, image=path)
TL1 = Text(app, text="[1]=Inactivo")
TL2 = Text(app, text="[2]=Explosión")
TL3 = Text(app, text="[3]=Fumarola")
TL4 = Text(app, text="[4]=Explosión Y Fumarola")
TL4 = Text(app, text="[5]=No sé")
TL5 = Text(app, text=f"Modelo recomienda: {row['PredictedClass']} ({row['Certainty']:.02f}% seguro) | Actual: {row['RealClass']}")

classmap={
    1:"INAC",
    2:"EXP",
    3:"FUM",
    4:"FUM+EXP",
    5:"UNK",
}
Input= TextBox(app, width="fill", multiline=False)
#Input.bg="black"

Barrier=False
def textupdate():
    global Barrier, row, path
    if Barrier:
        return
    value=Input.value.strip()
    Input.value=value
    if len(value)==0 or value[-1]!=".":
        return
    Barrier=True
    
    class_=None
    try:
        num=int(value[:-1].strip())
        class_=classmap[num]
    except:
        Input.value=""
        Barrier=False
        return
    df.to_feather("NN_INPUT.backup.feather")
    set_iclass(df,row.name,class_)
    mcl_df.loc[row.name,"Written"]=1
    #set_iclass(df,row.name,filtered[0])
    df.to_feather("NN_INPUT.feather")
    print("SET",row.name,get_iclass(df,row.name),class_)
    while True:
        try:
            row,path=get_current_row()
            pic.image=path
            break
        except Exception as E:
            mcl_df.loc[row.name,"Written"]=1
            print("FOUND ERROR",E)
        
    TL5.clear()
    TL5.append(f"Modelo recomienda: {row['PredictedClass']} ({row['Certainty']:.02f}% seguro) | Actual: {row['RealClass']}")
    #print("NEXT:",row)
    #print("prev_y",df.iloc[current_image_idx,-2])
        
    #Load another image and reset the input
    #pic.image=get_next_image()

    app.title=f'{remaining_ct} to go.'
    Input.value=""
    Barrier=False
Input.update_command(textupdate)
Input.focus()

    
app.display()

/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]


SET 9577 EXP+FUM FUM+EXP


/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]


SET 18857 EXP+FUM FUM+EXP


/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]


...

/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]


SET 12260 UNK UNK


/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]


SET 478 UNK UNK


/tmp/ipykernel_7352/2071744375.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remaining=remaining[mcl_df["Written"]!=1]
